# 손글씨 데이터 정보 + OCR 모델 결과물 합치기

## 라이브러리

In [1]:
import pandas as pd
import numpy as np
import os
import json
import pprint
import glob

import tensorflow as tf
import cv2
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

import paddleocr
from paddleocr import PaddleOCR, draw_ocr
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

## 데이터 목록이 제대로 있는지 확인하는 코드

In [2]:
# JSON 파일 목록 정리
basic_path = 'data/HW_OCR/4.Validation/'
backgrounds = ['P.Paper/', 'T.Tablet/']
contents = ['R.Free/', 'O.Form/']
json_files_list = []
png_files_list = []

for background in backgrounds:
    for content in contents:
        tmp = []
        path = basic_path + background + content
        tmp = list(map(lambda x: path + str(x), filter(lambda x: x.endswith('.json'), os.listdir(path))))
        json_files_list.append(tmp)
        print('{}.json: {}개'.format(path, len(tmp)))
        
        tmp = list(map(lambda x: path + str(x), filter(lambda x: x.endswith('.png'), os.listdir(path))))
        png_files_list.append(tmp)
        print('{}.png: {}개'.format(path, len(tmp)))
        print()

data/HW_OCR/4.Validation/P.Paper/R.Free/.json: 35535개
data/HW_OCR/4.Validation/P.Paper/R.Free/.png: 35535개

data/HW_OCR/4.Validation/P.Paper/O.Form/.json: 43170개
data/HW_OCR/4.Validation/P.Paper/O.Form/.png: 43170개

data/HW_OCR/4.Validation/T.Tablet/R.Free/.json: 4789개
data/HW_OCR/4.Validation/T.Tablet/R.Free/.png: 4789개

data/HW_OCR/4.Validation/T.Tablet/O.Form/.json: 12525개
data/HW_OCR/4.Validation/T.Tablet/O.Form/.png: 12525개



In [3]:
cnt_P_AIHUB = 87205
cnt_T_AIHUB = 19314
cnt_AIHUB = cnt_P_AIHUB + cnt_T_AIHUB

P1 = len(json_files_list[0]) + len(json_files_list[1])
T1 = len(json_files_list[2]) + len(json_files_list[3])
print('갖고 있는 json 데이터 목록')
print('Paper: {}, Table: {}, total: {}'.format(P1, T1, P1+T1))
print('데이터 사이트와 내가 갖고 있는 데이터 목록 갯수와의 차이')
print('Paper: {}, Table: {}, total: {}'.format(cnt_P_AIHUB - P1, cnt_T_AIHUB - T1, cnt_AIHUB - (P1+T1)))

print()

P2 = len(png_files_list[0]) + len(png_files_list[1])
T2 = len(png_files_list[2]) + len(png_files_list[3])
print('갖고 있는 png 데이터 목록')
print('Paper: {}, Table: {}, total: {}'.format(P2, T2, P2+T2))
print('데이터 사이트와 내가 갖고 있는 데이터 목록 갯수와의 차이')
print('Paper: {}, Table: {}, total: {}'.format(cnt_P_AIHUB - P2, cnt_T_AIHUB - T2, cnt_AIHUB - (P2+T2)))

갖고 있는 json 데이터 목록
Paper: 78705, Table: 17314, total: 96019
데이터 사이트와 내가 갖고 있는 데이터 목록 갯수와의 차이
Paper: 8500, Table: 2000, total: 10500

갖고 있는 png 데이터 목록
Paper: 78705, Table: 17314, total: 96019
데이터 사이트와 내가 갖고 있는 데이터 목록 갯수와의 차이
Paper: 8500, Table: 2000, total: 10500


## 데이터 구조 확인

In [4]:
path = 'data/HW_OCR/4.Validation/P.Paper/R.Free/'
file = 'IMG_OCR_53_4PR_21045.json'
with open(path + file, 'r', encoding='UTF8') as f:
    data = json.load(f)
pprint.pprint(data)

{'Annotation': {'object_recognition': 0, 'text_language': 0},
 'Dataset': {'category': 0,
             'identifier': 'IMG_OCR_53',
             'label_path': 'HW_OCR/4.Validation/P.Paper/R.Free/',
             'name': '대용량 손글씨 데이터셋',
             'src_path': 'HW_OCR/4.Validation/P.Paper/R.Free/',
             'type': 1},
 'Images': {'acquistion_location': '자체',
            'application_field': '기타',
            'background': 0,
            'data_captured': '2021.09.06',
            'height': 3482,
            'identifier': 'IMG_OCR_53_4PR_21045',
            'media_type': 0,
            'pen_color': 'red',
            'pen_type': 0,
            'type': 'png',
            'width': 2469,
            'writer_age': 47,
            'writer_sex': 1,
            'written_content': 1},
 'bbox': [{'data': '021045',
           'id': 1,
           'x': [1857, 1857, 2262, 2262],
           'y': [153, 248, 153, 248]},
          {'data': '경상북도',
           'id': 2,
           'x': [347, 347, 746, 74

### 결과 데이터 구성 요소

- label에서 가져올 정보
    - id
        - Images > identifier
    - image
        - Images > identifer + '.' + type
        - 두 요소를 .으로 이어 붙어야 한다.
    - age
        - Images > writer_age
    - sex
        - Images > writer_sex
    - background
        - Images > background
        - 종이: 0
        - 타블렛: 1
    - content
        - Images > written_content
        - 정보 제공: 0
        - 자유 필사: 1
    - text_origin
        - bbox > data
    - bbox_origin
        - bbox > data, id, x, y 순서로 정렬
        - x와 y에 존재하는 숫자는 좌측 하부, 좌측 상부, 우측 하부, 우측 상부 순서로 정렬되어 있음
        - OCR 결과물과 동일한 순서로 작업을 할 계획이라
            - 좌측 하부, 우측 하부,우측 상부, 좌측 상부 순으로 [x, y] 묶음의 나열로 바꿀것

<br>

- OCR 모델에서 가져올 정보
    - text_ocr
        - ocr 모델이 찾은 모든 글씨
    - bbox_ocr
        - ocr 모델이 찾은 모든 글씨의 bbox 값
        - bbox값이 좌측 하부, 우측 하부,우측 상부, 좌측 상부 순으로 [x, y] 묶음의 나열로 있음
    - score_ocr
        - ocr 모델이 찾은 모든 글씨의 점수값
        
<br>

- label과 OCR 모델를 활용해 정합한 정보
    - text
        - 정답 데이터인 text_origin과 ocr 모델이 찾은 데이터인 text_ocr을 비교하여 찾은 데이터 중 정답만을 가져온 것
        - 교집합
    - bbox
        - text 컬럼에 대응되는 bbox값(bbox_ocr 컬럼에서 가져온 것)

## 결과 데이터 생성 함수

In [5]:
# 결과 데이터 컬럼 및 df 생성

col = ['id', 'image', 'age', 'sex', 'background', 'content',
       'text_origin', 'bbox_origin', 'text_ocr', 'bbox_ocr', 'score_ocr', 'text', 'bbox', 'score']

ocr_df = pd.DataFrame(columns = col)
ocr_df

,id,image,age,sex,background,content,text_origin,bbox_origin,text_ocr,bbox_ocr,score_ocr,text,bbox,score


In [6]:
# JSON 파일 로드 함수
def load_json(json_path):
    try:
        encoder = 'utf-8'
        with open(json_path, 'r', encoding = encoder) as f:
            data = json.load(f)
    except:
        try:
            encoder = 'cp949'
            with open(json_path, 'r', encoding = encoder) as f:
                data = json.load(f)
            
        except:
            encoder = 'euc-kr'
            with open(json_path, 'r', encoding = encoder) as f:
                data = json.load(f)
    
    return data

# JSON 파일 목록 정리
basic_path = 'data/HW_OCR/4.Validation/'
backgrounds = ['P.Paper/', 'T.Tablet/']
contents = ['R.Free/', 'O.Form/']
json_files_list = []
for background in backgrounds:
    tmp = []
    for content in contents:
        path = basic_path + background + content
        tmp = list(map(lambda x: path + str(x), filter(lambda x: x.endswith('.json'), os.listdir(path))))
        json_files_list += tmp
    
print(len(json_files_list))

96019


In [10]:
def jsonNocr2df(json_files_list):
    
    # json 데이터 가져오는 부분
    def json2var(json_path):

        # JSON 파일 변수
        data = load_json(json_path)

        # image 파일 경로 변수
        image_path = 'data/' + data['Dataset']['src_path'] + data['Images']['identifier'] + '.' + data['Images']['type']

        # 진행 상황 확인용 프린트문
#         print(image_path)

        # 각 단어의 바운딩 박스와 텍스트 정보
        texts_list = []
        bounding_box_list = []

        # 전처리된 이미지 리스트
        for word in data['bbox']:
            text = word['data']
            texts_list.append(text)

            # 진행 상황 확인용 프린트문
#             print(text)

            # 좌표들 순서 위치            # paddle_ocr bbox 결과 좌표 순서 위치
            # 1    3                    # 3    2
            # 0    2                    # 0    1
            bounding_box = [[word['x'][0], word['y'][0]],
                            [word['x'][2], word['y'][2]],
                            [word['x'][3], word['y'][3]],
                            [word['x'][1], word['y'][1]]
                           ]
            bounding_box_list.append(bounding_box)

            # 진행 상황 확인용 프린트문
#             print(bounding_box)
        df_id = data['Images']['identifier']
        df_image = image_path
        df_age = data['Images']['writer_age']
        df_sex = data['Images']['writer_sex']
        df_background = data['Images']['background']
        df_content = data['Images']['written_content']
        df_text_origin = texts_list
        df_bbox_origin = bounding_box_list
        
        return df_id, df_image, df_age, df_sex, df_background, df_content, df_text_origin, df_bbox_origin
    
    # ocr 모델 적용부분
    def png2ocr(img_path):
        ocr_model = PaddleOCR(lang = 'korean')
        result = ocr_model.ocr(img_path, cls = False)

        return result

    # ocr 모델 결과를 리스트 변수로 각각 변환
    def ocr_TSB(ocr_result):
        res = ocr_result[0]

        texts_ocr = [res[i][1][0] for i in range(len(res))]
        scores_ocr = [float(res[i][1][1]) for i in range(len(res))]
        boxes_ocr = [res[i][0] for i in range(len(res))]

        return texts_ocr, scores_ocr, boxes_ocr

    # 정답데이터와 ocr 모델 결과데이터를 비교하여 교집합 내용만 고르는 코드
    def ocrINTERcorrect(correct, texts, scores, boxes):

        texts_inter = []
        scores_inter = []
        boxes_inter = []

        for t, s, b in zip(texts, scores, boxes):

            for c in correct:
                if t == c:
                    texts_inter.append(t)
                    scores_inter.append(s)
                    boxes_inter.append(b)
        if len(texts_inter) == len(scores_inter) == len(boxes_inter):
            return texts_inter, scores_inter, boxes_inter

        else:
            print('ERROR CHECK the CODE')

            print(texts_inter)
            print(len(texts_inter), len(scores_inter), len(boxes_inter))
            
            
    # json 파일마다 필요한 부분 적용
    for i, json_path in enumerate(json_files_list):

        # 진행 상황 확인용 프린트문
        if i % 10 == 0:
            print()
            print('<<{}번째>>'.format(i))
            print()

        # df에 들어갈 전체 변수 선언 부분
        # json 파트
        df_id, df_image, df_age, df_sex, df_background, df_content, df_text_origin, df_bbox_origin = json2var(json_path)

        # ocr 파트
        df_text_ocr, df_score_ocr, df_bbox_ocr = ocr_TSB(png2ocr(df_image))
        df_text, df_score, df_bbox = ocrINTERcorrect(df_text_origin, df_text_ocr, df_score_ocr, df_bbox_ocr)

        ocr_df.loc[len(ocr_df)] = [df_id, df_image, df_age, df_sex, df_background, df_content,
                                   df_text_origin, df_bbox_origin,
                                   df_text_ocr, df_bbox_ocr, df_score_ocr, df_text, df_bbox, df_score]


## 샘플 확인

In [11]:
jsonNocr2df(json_files_list[:10])


<<0번째>>

[2024/08/29 01:34:41] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max',

[2024/08/29 01:35:14] ppocr DEBUG: dt_boxes num : 69, elapsed : 0.1498572826385498
[2024/08/29 01:35:23] ppocr DEBUG: rec_res num  : 69, elapsed : 8.583782434463501
[2024/08/29 01:35:23] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='.

[2024/08/29 01:35:58] ppocr DEBUG: dt_boxes num : 71, elapsed : 0.3945643901824951
[2024/08/29 01:36:06] ppocr DEBUG: rec_res num  : 71, elapsed : 8.639906644821167
[2024/08/29 01:36:07] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='.

[2024/08/29 01:36:42] ppocr DEBUG: dt_boxes num : 74, elapsed : 0.18585515022277832
[2024/08/29 01:36:54] ppocr DEBUG: rec_res num  : 74, elapsed : 12.526983737945557
[2024/08/29 01:36:55] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir=

In [12]:
ocr_df.tail()

,id,image,age,sex,background,content,text_origin,bbox_origin,text_ocr,bbox_ocr,score_ocr,text,bbox,score
5,IMG_OCR_53_4PR_04363,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,37,0,0,1,"[004363, 대구광역시, 서울특별시, 전라남도, 경기도, 광산구, 창원시, 마산...","[[[1939, 147], [2225, 147], [2225, 228], [1939...","[No, 004303, 손글씨OCR, 자유필사, DataSheet, 00200500...","[[[1764.0, 167.0], [1832.0, 167.0], [1832.0, 2...","[0.9999909996986389, 0.7924535870552063, 0.997...","[제곧내, EMS]","[[[1405.0, 1887.0], [1655.0, 1901.0], [1650.0,...","[0.8001788258552551, 0.7710990309715271]"
6,IMG_OCR_53_4PR_04364,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,37,0,0,1,"[004364, 경기도, 대전광역시, 강원도, 광주광역시, 과천시, 서구, 거창군,...","[[[1901, 135], [2214, 135], [2214, 221], [1901...","[004364, No, 손글씨OCR, 자유필사, DataSheet, |, 00200...","[[[1903.0, 142.0], [2202.0, 133.0], [2204.0, 2...","[0.9245392680168152, 0.9999925494194031, 0.997...","[004364, NPS, IC, 연신이]","[[[1903.0, 142.0], [2202.0, 133.0], [2204.0, 2...","[0.9245392680168152, 0.8990578651428223, 0.746..."
7,IMG_OCR_53_4PR_04365,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,37,0,0,1,"[004365, 세종특별시, 울산광역시, 강원도, 제주특별자치도, 유성구, 관악구,...","[[[1914, 134], [2205, 134], [2205, 225], [1914...","[[365, No, 00, 손글씨OCR, 자유필사, DataSheet, O02g0s...","[[[2000.0, 145.0], [2194.0, 136.0], [2197.0, 2...","[0.6365471482276917, 0.9999923706054688, 0.987...",[],[],[]
8,IMG_OCR_53_4PR_04366,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,37,0,0,1,"[004366, 경상남도, 경기도, 제주특별자치도, 서울특별시, 남구, 장수군, 청...","[[[1898, 144], [2171, 144], [2171, 226], [1898...","[No, 4766, 손글씨OCR, 자유필사, DataSheet, -, O02G 99...","[[[1761.0, 163.0], [1835.0, 163.0], [1835.0, 2...","[0.9999943971633911, 0.8207401037216187, 0.998...","[신전동, 핑프, ICT, 변번세]","[[[388.0, 1674.0], [580.0, 1674.0], [580.0, 17...","[0.7397688031196594, 0.7042297124862671, 0.867..."
9,IMG_OCR_53_4PR_04367,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,37,0,0,1,"[004367, 서울특별시, 울산광역시, 전라북도, 강원도, 고성군, 관악구, 창원...","[[[1910, 145], [2188, 145], [2188, 227], [1910...","[No, 00, 4264, 손글씨OCR, 자유필사, DataSheet, 000009...","[[[1767.0, 167.0], [1835.0, 167.0], [1835.0, 2...","[0.9999910593032837, 0.9937096834182739, 0.830...","[관악구, 신용동, 방제인, 강용방]","[[[928.0, 1239.0], [1131.0, 1239.0], [1131.0, ...","[0.7158606648445129, 0.7330688834190369, 0.998..."


In [13]:
ocr_df.to_csv('df_json+ocr_sample10.csv', index = False)

## 전체 적용
- for문으로 오류가 나도 중간 저장이 될 수 있도록 코드를 분배할 것
- 아래부터는 너무 졸려서 내일 할것

In [ ]:
for cnt in range(len(json_files_list)):
    
    start = 
    if cnt 
    
    jsonNocr2df(json_files_list[:])
    
    if len(ocr_df) == :
        ocr_df.to_csv('result_df_sample.csv', index = False)

In [ ]:
# def open_csv(country = 'korea, south'):
    
#     # 변수명 리스트
#     varList = []
    
#     # COUNTRY NAME에 country 인수가 포함되어있는 행을 추출
#     csvList = dfFiletoCountry[dfFiletoCountry['COUNTRY NAME'].str.contains((country.upper())) == True]
#     # 국가별 기상 관측 장소 갯수
#     cnt = len(csvList)
#     # 추출된 행의 RealFileNM값들을 리스트로 저장
#     stnList = csvList['RealFileNM'].values
    
#     # 
#     for STN in stnList:
#         varNM = 'df_{}'.format(STN)

#         globals()[varNM] = pd.read_csv('data_climate/gsom-latest/' + STN)
#         varList.append(varNM)
    
#     print('변수 갯수: ', cnt)
          
#     return varList

# 필요시 추가 작업

# 업무 C에 적용하기 위한 DF

In [ ]:
ocr_df.head()

# 업무 D에 적용하기 위한 DF

In [ ]:
ocr_df.head()

# 결과 시각화

In [ ]:
def visualizeOCR(df):
    
    img_path = df['image']
    
    boxes = df['bbox_ocr']
    texts = df['text_ocr']
    scores = df['score_ocr']
    

    # Specifying font path for draw_ocr method
    font_path = 'C:/Users/UserK/ds_study/DL_proj/PaddleOCR-main/doc/fonts/korean.TTF'

    # imports image
    img = cv2.imread(img_path) 

    # reorders the color channelsa
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

    # Visualize our image and detections
    # resizing display area
    plt.figure(figsize = (65, 65))

    # draw annotations on image
    annotated = draw_ocr(img, boxes, texts, scores, font_path = font_path)

    # show the image using matplotlib
    plt.imshow(annotated)